# Vector Databases - Research

Vector search is a cutting-edge approach to searching and retrieving data that leverages the power of vector similarity calculations. Unlike traditional keyword-based search, which matches documents based on the occurrence of specific terms, vector search focuses on the semantic meaning and similarity of data points. By representing data as vectors in a high-dimensional space, vector search enables more accurate and intuitive search results.

In [ ]:
pip install python-dotenv==1.0.1 langchain==0.2.1 langchain-community==0.2.1 pypdf==4.2.0

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

First, we need to load a document. Let's try with a markdown file and a pdf.

In [9]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
# Markdown

In [12]:
# PDF

from langchain_community.document_loaders import PyPDFLoader

pdf_loader = PyPDFLoader("example_data/example_pdf.pdf")

# Define how the text should be split:
#  - Each chunk should be up to 512 characters long.
#  - There should be an overlap of 64 characters between consecutive chunks. This overlap helps maintain context across the chunks.
splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=64)

# Load pdf and split into chunks.
pdf_chunks = pdf_loader.load_and_split(text_splitter=splitter)

In [13]:
pdf_chunks

[Document(page_content='GENERATIVE AI: HYPE,  OR \nTRUL\nY TRANSFORMATIVE?ISSUE 120 | July 5, 2023 | 12:28 PM EDT”\x01\x01\x01\x01\x01P\x01\x01\x01\x01\x01\x01\x01\x01\x01 \x01\x01 \x01 \x01Global Macro  \nResearch\nInvestors should consider this report as only a single factor in making their investment decision. For \nReg AC certiﬁcation and other important disclosures, see the Disclosure Appendix, or go to www.gs.com/research/hedge.html.\nThe Goldman Sachs Group, Inc.Since the release of OpenAI’s generative AI tool ChatGPT in November, investor', metadata={'source': 'example_data/example_pdf.pdf', 'page': 0}),
 Document(page_content='interest in generative AI technology has surged. The disruptive potential of  this technology, and whether the hype around it—and market pricing—has gone too far, is Top of Mind. We speak with Conviction’s Sarah Guo, NYU’s Gary Marcus, and GS GIR’s US software and internet analysts Kash Rangan and Eric Sheridan about what the technology can—and can’t—do 

# Astra DB